In [ ]:
#!pip install pymorphy2

In [14]:
import pickle
import nltk
import pymorphy2
import re
import json
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('russian')
stop_words.extend(['который', 'https', 'также', 'другой', 'которые', 'которым', 'которых', 'которая', 'которому', 'каждый'])
morph = pymorphy2.MorphAnalyzer()

In [ ]:
total_last_week = pd.read_csv("/content/total_last_week.csv", sep = '\t', error_bad_lines=False)

In [21]:
with open('/content/frequency_dict_half_of_the_year.pkl', 'rb') as f:
    frequency_dict_half_of_the_year = pickle.load(f)

with open('/content/frequency_dict_last_week.pkl', 'rb') as f:
    frequency_dict_last_week = pickle.load(f)

In [43]:
general_dict = dict()
for key, value in frequency_dict_last_week.items():
  if key in frequency_dict_half_of_the_year:
    general_dict[key] = value / frequency_dict_half_of_the_year[key]
sorted_general_dict = sorted(general_dict.items(), key=lambda x: x[1], reverse = True)[:3]

In [44]:
final_search = []
for elem in sorted_general_dict:
  final_search.append(elem[0])

In [45]:
def find_and_return_article():
  trend_result = []
  for trend_word in final_search:
    trend_word_dictionary = dict()
    for counter in range(total_last_week.shape[0]):
      text = total_last_week.MESSAGE[counter]
      if type(text) != float:
          tokens = nltk.word_tokenize(text)   
          filtered_text = [word.lower() for word in tokens if word.lower() not in stop_words] 
          final_text = []
          for word in filtered_text:
            if word.isalpha() and len(word) > 4:
              p = morph.parse(word)[0]
              final_text.append(p.normal_form)
            else:
              continue
          if trend_word in final_text:
            trend_word_dictionary[counter] = final_text.count(trend_word)
    trend_result.append(sorted(trend_word_dictionary.items(), key=lambda x: x[1], reverse = True)[0])
  return trend_result

In [46]:
trend_result = find_and_return_article()

In [28]:
general_trend_collection = []
for trend_index in trend_result:
  text = re.sub(' - РИА Новости', '', total_last_week.MESSAGE[trend_index[0]])
  text = re.sub(' – РИА Новости', '', text)
  general_trend_collection.append(text)

In [29]:
def write_to_json():
  items = []
  for index in range(len(general_trend_collection)):
    items.append(
          {
              "id": index,
              "text_of_the_article": general_trend_collection[index]
          }
      )
  with open('trend_news.json', 'w') as file:
    json.dump(items, file, indent=4, ensure_ascii=False)

In [30]:
write_to_json()